# Transformer Network Application: Named-Entity Recognition

Welcome to Week 4's first ungraded lab. In this notebook you'll explore one application of the transformer architecture that you built in the previous assignment.

**After this assignment you'll be able to**:

* Use tokenizers and pre-trained models from the HuggingFace Library.
* Fine-tune a pre-trained transformer model for Named-Entity Recognition

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Table of Contents

- [Packages](#0)
- [1 - Named-Entity Recogniton to Process Resumes](#1)
    - [1.1 - Data Cleaning](#1-1)
    - [1.2 - Padding and Generating Tags](#1-2)
    - [1.3 - Tokenize and Align Labels with 🤗 Library](#1-3)
        - [Exercise 1 - tokenize_and_align_labels](#ex-1)
    - [1.4 - Optimization](#1-4)

<a name='0'></a>
## Packages

Run the following cell to load the packages you'll need.

In [2]:
import pandas as pd
import tensorflow as tf
import json
import random
import logging
import re

<a name='1'></a>
## 1 - Named-Entity Recogniton to Process Resumes

When faced with a large amount of unstructured text data, named-entity recognition (NER) can help you detect and classify important information in your dataset. For instance, in the running example "Jane vists Africa in September", NER would help you detect "Jane", "Africa", and "September" as named-entities and classify them as person, location, and time.

* You will use a variation of the Transformer model you built in the last assignment to process a large dataset of resumes.
* You will find and classify relavent information such as the companies the applicant worked at, skills, type of degree, etc.

<a name='1-1'></a>
### 1.1 - Dataset Cleaning

In this assignment you will optimize a Transformer model on a dataset of resumes. Take a look at how the data you will be working with are structured.

In [3]:
df_data = pd.read_json("/content/drive/MyDrive/w4a2/ner.json", lines=True)
df_data = df_data.drop(['extras'], axis=1)
df_data['content'] = df_data['content'].str.replace("\n", " ")

In [4]:
df_data.head()

,content,annotation
0,Abhishek Jha Application Development Associate...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar Active member of IIIT Committee...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina Hyderabad, Telangana - E...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai Operational Analyst (SQL DBA) Eng...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan lecturer - oracle tutorials Mum...,"[{'label': ['Degree'], 'points': [{'start': 20..."


In [5]:
df_data.iloc[0]['annotation']

[{'label': ['Skills'],
  'points': [{'start': 1295,
    'end': 1621,
    'text': '\n• Programming language: C, C++, Java\n• Oracle PeopleSoft\n• Internet Of Things\n• Machine Learning\n• Database Management System\n• Computer Networks\n• Operating System worked on: Linux, Windows, Mac\n\nNon - Technical Skills\n\n• Honest and Hard-Working\n• Tolerant and Flexible to Different Situations\n• Polite and Calm\n• Team-Player'}]},
 {'label': ['Skills'],
  'points': [{'start': 993,
    'end': 1153,
    'text': 'C (Less than 1 year), Database (Less than 1 year), Database Management (Less than 1 year),\nDatabase Management System (Less than 1 year), Java (Less than 1 year)'}]},
 {'label': ['College Name'],
  'points': [{'start': 939, 'end': 956, 'text': 'Kendriya Vidyalaya'}]},
 {'label': ['College Name'],
  'points': [{'start': 883, 'end': 904, 'text': 'Woodbine modern school'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 856, 'end': 860, 'text': '2017\n'}]},
 {'label': ['College 

In [6]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)
    return merged

In [7]:
def get_entities(df):

    entities = []

    for i in range(len(df)):
        entity = []

        for annot in df['annotation'][i]:
            try:
                ent = annot['label'][0]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass

        entity = mergeIntervals(entity)
        entities.append(entity)

    return entities

In [8]:
df_data['entities'] = get_entities(df_data)
df_data.head()

,content,annotation,entities
0,Abhishek Jha Application Development Associate...,"[{'label': ['Skills'], 'points': [{'start': 12...","[(0, 12, Name), (13, 46, Designation), (49, 58..."
1,Afreen Jamadar Active member of IIIT Committee...,"[{'label': ['Email Address'], 'points': [{'sta...","[(0, 14, Name), (62, 68, Location), (104, 148,..."
2,"Akhil Yadav Polemaina Hyderabad, Telangana - E...","[{'label': ['Skills'], 'points': [{'start': 37...","[(0, 21, Name), (22, 31, Location), (65, 117, ..."
3,Alok Khandai Operational Analyst (SQL DBA) Eng...,"[{'label': ['Skills'], 'points': [{'start': 80...","[(0, 12, Name), (13, 51, Designation), (54, 60..."
4,Ananya Chavan lecturer - oracle tutorials Mum...,"[{'label': ['Degree'], 'points': [{'start': 20...","[(0, 13, Name), (14, 22, Designation), (24, 41..."


In [9]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    #only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']

                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1 , label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [10]:
data = trim_entity_spans(convert_dataturks_to_spacy("/content/drive/MyDrive/w4a2/ner.json"))

In [11]:
!pip install pandas
import pandas as pd
from tqdm.notebook import tqdm

def clean_dataset(data):
    # Instead of appending rows iteratively, collect them in a list
    all_rows = []
    sum1 = 0
    for i in tqdm(range(len(data))):
        start = 0
        emptyList = ["Empty"] * len(data[i][0].split())
        numberOfWords = 0
        lenOfString = len(data[i][0])
        strData = data[i][0]
        strDictData = data[i][1]
        lastIndexOfSpace = strData.rfind(' ')
        for i in range(lenOfString):
            if (strData[i]==" " and strData[i+1]!=" "):
                for k,v in strDictData.items():
                    for j in range(len(v)):
                        entList = v[len(v)-j-1]
                        if (start>=int(entList[0]) and i<=int(entList[1])):
                            emptyList[numberOfWords] = entList[2]
                            break
                        else:
                            continue
                start = i + 1
                numberOfWords += 1
            if (i == lastIndexOfSpace):
                for j in range(len(v)):
                        entList = v[len(v)-j-1]
                        if (lastIndexOfSpace>=int(entList[0]) and lenOfString<=int(entList[1])):
                            emptyList[numberOfWords] = entList[2]
                            numberOfWords += 1
        # Append the row to the list
        all_rows.append([emptyList])
        sum1 = sum1 + numberOfWords
    # Create the DataFrame from the list of rows
    cleanedDF = pd.DataFrame(all_rows, columns=["setences_cleaned"])
    return cleanedDF

In [12]:
cleanedDF = clean_dataset(data)

  0%|          | 0/220 [00:00<?, ?it/s]

Take a look at your cleaned dataset and the categories the named-entities are matched to, or 'tags'.

In [13]:
cleanedDF.head()

,setences_cleaned
0,"[Name, Name, Designation, Designation, Designa..."
1,"[Name, Name, Empty, Empty, Empty, Empty, Empty..."
2,"[Name, Name, Name, Empty, Empty, Empty, Empty,..."
3,"[Name, Name, Designation, Designation, Designa..."
4,"[Name, Name, Designation, Empty, Companies wor..."


<a name='1-2'></a>
### 1.2 - Padding and Generating Tags

Now, it is time to generate a list of unique tags you will match the named-entities to.

In [14]:
unique_tags = set(cleanedDF['setences_cleaned'].explode().unique())#pd.unique(cleanedDF['setences_cleaned'])#set(tag for doc in cleanedDF['setences_cleaned'].values.tolist() for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [15]:
unique_tags

{'College Name',
 'Companies worked at',
 'Degree',
 'Designation',
 'Email Address',
 'Empty',
 'Graduation Year',
 'Location',
 'Name',
 'Skills',
 'UNKNOWN',
 'Years of Experience'}

Next, you will create an array of tags from your cleaned dataset. Oftentimes your input sequence will exceed the maximum length of a sequence your network can process. In this case, your sequence will be cut off, and you need to append zeroes onto the end of the shortened sequences using this [Keras padding API](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences).

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
MAX_LEN = 512
labels = cleanedDF['setences_cleaned'].values.tolist()

tags = pad_sequences([[tag2id.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2id["Empty"], padding="post",
                     dtype="long", truncating="post")

In [18]:
tags

array([[1, 1, 6, ..., 8, 8, 8],
       [1, 1, 8, ..., 8, 8, 8],
       [1, 1, 1, ..., 8, 4, 8],
       ...,
       [1, 1, 6, ..., 8, 8, 8],
       [1, 1, 6, ..., 8, 8, 8],
       [1, 1, 6, ..., 8, 8, 8]])

<a name='1-3'></a>
### 1.3 - Tokenize and Align Labels with 🤗 Library

Before feeding the texts to a Transformer model, you will need to tokenize your input using a [🤗 Transformer tokenizer](https://huggingface.co/transformers/main_classes/tokenizer.html). It is crucial that the tokenizer you use must match the Transformer model type you are using! In this exercise, you will use the 🤗 [DistilBERT fast tokenizer](https://huggingface.co/transformers/model_doc/distilbert.html), which standardizes the length of your sequence to 512 and pads with zeros. Notice this matches the maximu length you used when creating tags.

In [19]:
from transformers import DistilBertTokenizerFast #, TFDistilBertModel
tokenizer = DistilBertTokenizerFast.from_pretrained('/content/drive/MyDrive/w4a2/tokenizer/')

Transformer models are often trained by tokenizers that split words into subwords. For instance, the word 'Africa' might get split into multiple subtokens. This can create some misalignment between the list of tags for the dataset and the list of labels generated by the tokenizer, since the tokenizer can split one word into several, or add special tokens. Before processing, it is important that you align the lists of tags and the list of labels generated by the selected tokenizer with a `tokenize_and_align_labels()` function.

<a name='ex-1'></a>
### Exercise 1 - tokenize_and_align_labels

Implement `tokenize_and_align_labels()`. The function should perform the following:
* The tokenizer cuts sequences that exceed the maximum size allowed by your model with the parameter `truncation=True`
* Aligns the list of tags and labels with the tokenizer `word_ids` method returns a list that maps the subtokens to the original word in the sentence and special tokens to `None`.
* Set the labels of all the special tokens (`None`) to -100 to prevent them from affecting the loss function.
* Label of the first subtoken of a word and set the label for the following subtokens to -100.

In [20]:
label_all_tokens = True
def tokenize_and_align_labels(tokenizer, examples, tags):
    tokenized_inputs = tokenizer(examples, truncation=True, is_split_into_words=False, padding='max_length', max_length=512)
    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Now that you have tokenized inputs, you can create train and test datasets!

In [21]:
test = tokenize_and_align_labels(tokenizer, df_data['content'].values.tolist(), tags)
train_dataset = tf.data.Dataset.from_tensor_slices((
    test['input_ids'],
    test['labels']
))

In [22]:
test['labels'][0]

[-100,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 6,
 6,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 5,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 6,
 6,
 8,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 6,
 6,
 6,
 6,
 6,
 8,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 10,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 10,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,

<a name='1-4'></a>
### 1.4 - Optimization

Fantastic! Now you can finally feed your data into into a pretrained 🤗 model. You will optimize a DistilBERT model, which matches the tokenizer you used to preprocess your data. Try playing around with the different hyperparamters to improve your results!

In [23]:
from transformers import TFDistilBertForTokenClassification

model = TFDistilBertForTokenClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_tags))
model.save_pretrained('/content/drive/MyDrive/w4a2/model/')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

In [24]:
!pip install transformers tensorflow
from transformers import TFDistilBertForTokenClassification, create_optimizer
import tensorflow as tf

In [25]:
num_train_steps = len(train_dataset) * 3  # Assuming 3 epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=1e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps
)

# Compile the model
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Epoch 1/3


AttributeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_tf_utils.py", line 1588, in compute_loss  *
        return super().compute_loss(*args, **kwargs)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1206, in compute_loss  **
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/compile_utils.py", line 275, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/compile_utils.py", line 854, in match_dtype_and_rank
        if (y_t.dtype.is_floating and y_p.dtype.is_floating) or (

    AttributeError: 'NoneType' object has no attribute 'dtype'


wandb

In [26]:
pip install wandb

In [27]:
import wandb
wandb.init(project='tf-distl-bert-token-cls', name='neural-net')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!pip install --upgrade wandb

In [36]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [45]:
from transformers import TFDistilBertForTokenClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Initialize W&B
wandb.init(project="dist-tf-bert-token-cls", entity="tuanngoc")

# Set up WandbCallback with custom options
wandb_callback = wandb.keras.WandbCallback(
    monitor="val_accuracy",  # track validation accuracy
    log_weights=True,        # log model weights
    log_gradients=True,      # log gradients
    save_model=True,         # save the model with the best val_accuracy
    log_batch_frequency=10,  # log every 10 batches
    validation_data=test,  # validation data to visualize
    training_data=train_dataset.shuffle(1000).batch(16).take(1) # Take a small sample for efficiency
)


num_train_steps = len(train_dataset) * 3  # Assuming 3 epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=1e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps
)

# Define the number of labels
num_labels = len(unique_tags)  # Replace unique_tags with your list of tags

# Load the model with the specified number of labels
model = TFDistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16, callbacks=[wandb.keras.WandbCallback()])


accuracy,█▁▁
epoch,▁▅█
accuracy,0.00531
epoch,2
loss,nan


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

Epoch 1/3
14/14 [==============================] - 31s 799ms/step - loss: nan - accuracy: 0.0070
Epoch 2/3
14/14 [==============================] - 11s 796ms/step - loss: nan - accuracy: 0.0053
Epoch 3/3
14/14 [==============================] - 11s 781ms/step - loss: nan - accuracy: 0.0053


In [ ]:
import os
import tensorflow as tf

# Define the custom callback for saving models and logging
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Log epoch completion and any other metrics
        print(f"Epoch {epoch+1} finished with logs: {logs}")

        # Define directory and filename format for saving the model
        save_dir = "/content/drive/MyDrive/w4a2/model"
        os.makedirs(save_dir, exist_ok=True)

        # Use validation accuracy if available, otherwise fallback to training accuracy
        val_acc = logs.get("val_accuracy")
        train_acc = logs.get("accuracy")
        if val_acc:
            model_name = f"model_val_acc_{val_acc:.4f}"
        else:
            model_name = f"model_train_acc_{train_acc:.4f}"

        # Save the model in SavedModel format
        model_path = os.path.join(save_dir, model_name)
        self.model.save(model_path, save_format="tf")
        print(f"Model saved to {model_path}")

    # Adding the following hook implementations
    def _implements_train_batch_hooks(self):
        return False

    def _implements_test_batch_hooks(self):
        return False

    def _implements_predict_batch_hooks(self):
        return False

# Use the custom callback in model.fit
model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=10,
    callbacks=[CustomCheckpointCallback()]
)


In [ ]:
import os
import tensorflow as tf
from transformers import DistilBertTokenizer

# Define the custom callback for saving models and logging
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Log epoch completion and any other metrics
        print(f"Epoch {epoch+1} finished with logs: {logs}")

        # Define directory and filename format for saving the model
        save_dir = "/content/drive/MyDrive/w4a2/model"
        os.makedirs(save_dir, exist_ok=True)

        # Use validation accuracy if available, otherwise fallback to training accuracy
        val_acc = logs.get("val_accuracy")
        train_acc = logs.get("accuracy")
        if val_acc:
            model_name = f"model_val_acc_{val_acc:.4f}"
        else:
            model_name = f"model_train_acc_{train_acc:.4f}"

        # Save the model in SavedModel format
        model_path = os.path.join(save_dir, model_name)
        self.model.save(model_path, save_format="tf")
        print(f"Model saved to {model_path}")

    # Adding the following hook implementations
    def _implements_train_batch_hooks(self):
        return False

    def _implements_test_batch_hooks(self):
        return False

    def _implements_predict_batch_hooks(self):
        return False

# Initialize the tokenizer (assuming a DistilBERT model)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Prepare a sample input using the tokenizer
sample_text = "This is a sample input to initialize the model."
sample_input = tokenizer(sample_text, return_tensors="tf")

# Run a forward pass to initialize the model layers
model(sample_input)

# Train the model with the custom callback
model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=10,
    callbacks=[CustomCheckpointCallback()]
)

store weight after each epoch

In [ ]:
import os
import tensorflow as tf
from transformers import DistilBertTokenizer
from datetime import datetime

# Define the custom callback for saving models and logging
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Log epoch completion and any other metrics
        print(f"Epoch {epoch+1} finished with logs: {logs}")

        # Define directory and filename format for saving the model
        save_dir = "/content/drive/MyDrive/w4a2/model"
        os.makedirs(save_dir, exist_ok=True)

        # Use validation accuracy if available, otherwise fallback to training accuracy
        val_acc = logs.get("val_accuracy")
        train_acc = logs.get("accuracy")

        # Generate a unique model name using accuracy, epoch, and timestamp
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")  # Format timestamp for uniqueness
        if val_acc:
            model_name = f"model_epoch_{epoch+1}_val_acc_{val_acc:.4f}_{timestamp}"
        else:
            model_name = f"model_epoch_{epoch+1}_train_acc_{train_acc:.4f}_{timestamp}"

        # Save the model in SavedModel format with the unique name
        model_path = os.path.join(save_dir, model_name)
        self.model.save(model_path, save_format="tf")
        print(f"Model saved to {model_path}")

    # Adding the following hook implementations
    def _implements_train_batch_hooks(self):
        return False

    def _implements_test_batch_hooks(self):
        return False

    def _implements_predict_batch_hooks(self):
        return False

# Initialize the tokenizer (assuming a DistilBERT model)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Prepare a sample input using the tokenizer
sample_text = "This is a sample input to initialize the model."
sample_input = tokenizer(sample_text, return_tensors="tf")

# Run a forward pass to initialize the model layers
model(sample_input)

# Train the model with the custom callback
model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=10,
    callbacks=[CustomCheckpointCallback()]
)


store best weight

In [ ]:
import os
import tensorflow as tf
from transformers import DistilBertTokenizer

# Define the custom callback for saving only the best model at the end of training
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_dir):
        super().__init__()
        self.best_val_acc = -1  # Track the best validation accuracy
        self.best_weights = None  # Store the best model weights
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        # Get the current validation accuracy
        val_acc = logs.get("val_accuracy")

        # Update the best weights if current validation accuracy is the best so far
        if val_acc and val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.best_weights = self.model.get_weights()  # Save the best weights in memory
            print(f"Best validation accuracy improved to {val_acc:.4f} at epoch {epoch+1}")

    def on_train_end(self, logs=None):
        # Save the best model weights to disk at the end of training
        if self.best_weights:
            self.model.set_weights(self.best_weights)  # Set model to best weights
            model_name = f"best_model_val_acc_{self.best_val_acc:.4f}.tf"
            model_path = os.path.join(self.save_dir, model_name)
            self.model.save(model_path, save_format="tf")
            print(f"Best model saved to {model_path} with val_acc: {self.best_val_acc:.4f}")

    # Add the following hook implementations to avoid AttributeError
    def _implements_train_batch_hooks(self):
        return False

    def _implements_test_batch_hooks(self):
        return False

    def _implements_predict_batch_hooks(self):
        return False

# Initialize the tokenizer (assuming a DistilBERT model)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Prepare a sample input using the tokenizer
sample_text = "This is a sample input to initialize the model."
sample_input = tokenizer(sample_text, return_tensors="tf")

# Run a forward pass to initialize the model layers
model(sample_input)

# Directory where the best model will be saved
save_dir = "/content/drive/MyDrive/w4a2/model"

# Train the model with the custom callback to save only the best model at the end of training
model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=10,
    callbacks=[CustomCheckpointCallback(save_dir)]
)


check

In [ ]:
!ls /content/drive/MyDrive/w4a2/model

log file and path of recent best weight storing

In [ ]:
from transformers import TFDistilBertForTokenClassification, DistilBertTokenizer
import tensorflow as tf
import os

# Load the model
model = TFDistilBertForTokenClassification.from_pretrained('/content/drive/MyDrive/w4a2/model/')

# Define the custom callback for saving only the best model at the end of training
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_dir):
        super().__init__()
        self.best_val_acc = -1  # Track the best validation accuracy
        self.best_weights = None  # Store the best model weights
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        # Get the current validation accuracy
        val_acc = logs.get("val_accuracy")

        # Update the best weights if current validation accuracy is the best so far
        if val_acc and val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.best_weights = self.model.get_weights()  # Save the best weights in memory
            print(f"Best validation accuracy improved to {val_acc:.4f} at epoch {epoch+1}")

    def on_train_end(self, logs=None):
        # Save the best model weights to disk at the end of training
        if self.best_weights:
            self.model.set_weights(self.best_weights)  # Set model to best weights
            model_name = f"best_model_val_acc_{self.best_val_acc:.4f}.tf"
            model_path = os.path.join(self.save_dir, model_name)
            self.model.save(model_path, save_format="tf")

            # Log the file name and path of the recent file storing the best model
            print(f"Best model saved to {model_path} with val_acc: {self.best_val_acc:.4f}")
            print(f"File name: {model_name}")
            print(f"File path: {model_path}")

    # Add the following hook implementations to avoid AttributeError
    def _implements_train_batch_hooks(self):
        return False

    def _implements_test_batch_hooks(self):
        return False

    def _implements_predict_batch_hooks(self):
        return False

# Initialize the tokenizer (assuming a DistilBERT model)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Prepare a sample input using the tokenizer
sample_text = "This is a sample input to initialize the model."
sample_input = tokenizer(sample_text, return_tensors="tf")

# Run a forward pass to initialize the model layers
model(sample_input)

# Directory where the best model will be saved
save_dir = "/content/drive/MyDrive/w4a2/model"

# Train the model with the custom callback to save only the best model at the end of training
model.fit(
    train_dataset.shuffle(1000).batch(16),
    epochs=10,
    callbacks=[CustomCheckpointCallback(save_dir)]
)


remove

In [ ]:
!ls /content/drive/MyDrive/w4a2/model

In [ ]:
!rm -r /content/drive/MyDrive/w4a2/model/model_*

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy']) # can also use any keras loss fn
model.fit(train_dataset.shuffle(1000).batch(16),
          epochs=3,
          batch_size=16)

### Congratulations!

#### Here's what you should remember

- Named-entity recognition (NER) detects and classifies named-entities, and can help process resumes, customer reviews, browsing histories, etc.
- You must preprocess text data with the corresponding tokenizer to the pretrained model before feeding your input into your Transformer model